In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
import datetime
import string
from wordcloud import WordCloud, STOPWORDS
import re

In [ ]:
data = pd.read_csv('/kaggle/input/comcastcomplaints/comcast_fcc_complaints_2015.csv')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.info()
#There are no null values in the dataset

In [ ]:
data.describe(include='all')

In [ ]:
data.Date=pd.to_datetime(data.Date)
data['Date'] = pd.Series([val.date() for val in data['Date']])

In [ ]:
data.loc[data.Time=='10:58:37 P',"Time"] = '10:58:37 PM'

In [ ]:
data.Time=pd.to_datetime(data.Time,format='%I:%M:%S %p')
data['Time'] = pd.Series([val.time() for val in data['Time']])

In [ ]:
year = pd.Series([val.year for val in data.Date])
year.value_counts()

In [ ]:
#Creating a month  since we want the month wise distribution of complaints
data['Month'] = pd.Series([val.month for val in data['Date']])

In [ ]:
data['Month'].value_counts()

In [ ]:
data.groupby('Month').count()['Ticket #'].plot(kind='bar')
#June had the highest number of complaints

In [ ]:
data.groupby('Month').count()['Ticket #'].plot(kind='line',figsize=(10,5))
#Thus we can see that June saw a spike in the number of complaints

In [ ]:
data.groupby('Date').count()['Ticket #'].plot(kind='line',figsize=(15,10))
#We see the number of complaints were somewhat normal till 15 June but after that there was a massive jump.
#So, management should see what was the reason that in a span of around 10 days we could see such a jump in complaints.

In [ ]:
#Creating day column
data['Day']=pd.Series([val.day for val in data['Date']])

In [ ]:
data['Day'].value_counts()[0:10]
#We see that more complaints come in the second half of months.

In [ ]:
data.groupby(['Month','Day'])['Customer Complaint'].count().unstack()
#We can see the number of complaints for each day each month. 

In [ ]:
data['Customer Complaint'].value_counts()[0:50]
#These are the top 50 types of complaints with their respective frequencies.

In [ ]:
data['Status'].value_counts()

In [ ]:
text = " ".join(data['Customer Complaint'])
p=""
for char in text:
  if char not in string.punctuation:
    p=p+char
p=p.lower()
s = list(STOPWORDS)+['comcast']
wordcloud = WordCloud(background_color='white',random_state=10,stopwords=s,max_words=30,font_step=2,colormap='cool').generate(p)
plt.figure(figsize=(30,30))
plt.imshow(wordcloud)
plt.show()
#Dividing them into 5 major categories - Internet and Network Issue, Customer Servce, Billing, Financial, Other

In [ ]:
df = pd.DataFrame({'index':range(data.shape[0])})
data['Customer Complaint'] = data['Customer Complaint'].str.lower()
df['Customer_Service']=data['Customer Complaint'].str.extract("(service)")
df['Internet']=data['Customer Complaint'].str.extract("(internet)")
df['Billing']=data['Customer Complaint'].str.extract("(billing)")
df['Payment']=data['Customer Complaint'].str.extract("(charge)")
df['Cable']=data['Customer Complaint'].str.extract("(cable)")
df.shape
df.drop('index',axis=1,inplace=True)
s = df.notnull().sum().sort_values(ascending=False)
print(s)
#Hence we get the decreasing order of major complaint types

In [ ]:
data.loc[data.Status=='Solved','Status']='Closed'
data.loc[data.Status=='Pending','Status']='Open'

In [ ]:
#Plotting the number of open and closed cases state-wise
pd.crosstab(data.State,data.Status).plot(kind='bar',figsize=(20,15),stacked=True)
#We can see that Georgia has the highest number of unresolved complaints.

In [ ]:
Statewise = pd.crosstab(data.State,data.Status)
Statewise['Total']=Statewise.Open+Statewise.Closed
Statewise['Percentage_unresolved']=Statewise.Open/Statewise.Total*100
print(Statewise.Percentage_unresolved.sort_values(ascending=False))
#We can see that Kansas has the highest percentage of unresolved cases

Statewise['Percentage_solved']=Statewise.Closed/Statewise.Total*100
print(Statewise.Percentage_solved.sort_values(ascending=False))
#We can see that all the complaints are solved in Arkansas, District of Columbia, Iowa, Montana, 
#Nevada, New York, North Carolina, Ohio, Rhode Island.

In [ ]:
data.State.value_counts()
#Hence, we can see that Georgia has maximum complaints

In [ ]:
pd.crosstab(data['Received Via'],data['Status'])

In [ ]:
ser = data['Time']>datetime.time(12,00,00)

In [ ]:
data['Time_of_Day']=ser

In [ ]:
data['Time_of_Day']

In [ ]:
data['Time_of_Day']=data['Time_of_Day'].replace([True,False],['Evening','Morning'])

In [ ]:
data.Time_of_Day.value_counts().plot(kind='bar',figsize=(15,10))
#Hence, we see that more than double complaints are received at night.
#This might happen because families are back home in evenings and usage is higher which leads to lower speeds 
#and hence increased complaints
#We should consider providing higher bandwidths for evening time and compensate with slightly lower during the day. 

In [ ]:
pd.crosstab(data['Received Via'],data['Time_of_Day'])
#We should also provide better and more customer service representatives for evening time as complaints are higher.